# AVA Mind 

AvaMind is a personal digital angel that helps you change your life for the better.

It is based on AI analytics and techniques that have proved to be effective in current psychological and behavioral studies.

## Sentiment Analysis

In [1]:
import pandas as pd

In [2]:
users = ["Dima", "Alex", "Vlad"]
texts = ["I am feeling bad because of the war", "They are bombing my city", "We are winning in the sanctions war"]

In [3]:
df = pd.DataFrame({'User':users,'Text':texts})
df

,User,Text
0,Dima,I am feeling bad because of the war
1,Alex,They are bombing my city
2,Vlad,We are winning in the sanctions war


### English: TextBlob

In [4]:
df_english = df.copy()

In [5]:
from textblob import TextBlob

In [6]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
  
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
  
df_english['Subjectivity'] = df_english['Text'].apply(getSubjectivity)
df_english['Polarity'] = df_english['Text'].apply(getPolarity)

In [7]:
df_english

,User,Text,Subjectivity,Polarity
0,Dima,I am feeling bad because of the war,0.666667,-0.7
1,Alex,They are bombing my city,0.000000,0.0
2,Vlad,We are winning in the sanctions war,0.750000,0.5


- Subjectivity is from 0 (objectivity) to 1
- Polarity is from -1 to 1 (Negative, Netural and Positive)

In [8]:
def getAnalysis(Polarity):
    if Polarity < 0.0:
        return 'negative'
    elif Polarity == 0.0:
        return 'neutral'
    else:
        return 'positive'
    
df_english['Analysis'] = df_english['Polarity'].apply(getAnalysis)

In [9]:
df_english

,User,Text,Subjectivity,Polarity,Analysis
0,Dima,I am feeling bad because of the war,0.666667,-0.7,negative
1,Alex,They are bombing my city,0.000000,0.0,neutral
2,Vlad,We are winning in the sanctions war,0.750000,0.5,positive


### Russian: dostoevsky 0.6.0

https://github.com/bureaucratic-labs/dostoevsky

In [10]:
# pip install dostoevsky

CMD: python -m dostoevsky download fasttext-social-network-model

In [11]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
tokens = tokenizer.split('всё очень плохо')  # [('всё', None), ('очень', None), ('плохо', None)]

model = FastTextSocialNetworkModel(tokenizer=tokenizer)

messages = [
    'привет',
    'я люблю тебя!!',
    'малолетние дебилы'
]

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    # привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
    # люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
    # малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}]
    print(message, '->', sentiment)

привет -> {'speech': 1.0000100135803223, 'skip': 0.0020607432816177607}
я люблю тебя!! -> {'positive': 0.9886782765388489, 'skip': 0.005394937004894018}
малолетние дебилы -> {'negative': 0.9525841474533081, 'neutral': 0.13661839067935944}


### Google Translate: googletrans

- en: English
- ru: Russian
- uk: Ukrainian
- he: Hebrew

In [12]:
# pip install googletrans==3.1.0a0

In [13]:
df_translated = df.copy()

In [14]:
from googletrans import Translator
translator = Translator()
translation = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='uk')
print(translation.text)

Небо блакитне, а я люблю банани


In [15]:
def translate(text, dest):
    translator = Translator()
    translation = translator.translate(text, dest=dest)
    return translation.text

In [16]:
def translate_df(df, dest):
    translator = Translator()
    for column in df.columns:
        df[column] = df[column].apply(translate,dest=dest)
        translation = translator.translate(column, dest=dest)
        df.rename(columns={column:translation.text}, inplace=True)

In [17]:
translate_df(df_translated, 'ru')
df_translated

,Пользователь,Текст
0,Дима,Мне плохо из-за войны
1,Алекс,Они бомбят мой город
2,Влад,Мы побеждаем в санкционной войне


In [18]:
messages = df_translated['Текст']

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    print(message, '->', sentiment)

Мне плохо из-за войны -> {'negative': 0.9553291201591492, 'neutral': 0.08757384121417999}
Они бомбят мой город -> {'positive': 0.5544804334640503, 'neutral': 0.26285186409950256}
Мы побеждаем в санкционной войне -> {'neutral': 0.8558611869812012, 'positive': 0.21734751760959625}


## Convert all texts to English and use TextBlob

In [19]:
df_translated

,Пользователь,Текст
0,Дима,Мне плохо из-за войны
1,Алекс,Они бомбят мой город
2,Влад,Мы побеждаем в санкционной войне


In [20]:
translate_df(df_translated, 'en')
df_translated

,User,Text
0,Dima,I feel bad about the war
1,Alex,They bomb my city
2,Vlad,We are winning the sanctions war


In [21]:
df_translated['Subjectivity'] = df_translated['Text'].apply(getSubjectivity)
df_translated['Polarity'] = df_translated['Text'].apply(getPolarity)
df_translated['Analysis'] = df_translated['Polarity'].apply(getAnalysis)

In [22]:
df_translated

,User,Text,Subjectivity,Polarity,Analysis
0,Dima,I feel bad about the war,0.666667,-0.7,negative
1,Alex,They bomb my city,0.000000,0.0,neutral
2,Vlad,We are winning the sanctions war,0.750000,0.5,positive
